# Using PennyLane to build VQE algorithm for hidrogen molecule

## Comments

1. The function providing the Hartree-Fock solution is independent of the Hamiltonian. Hartree-Fock state is always just filled first n_electrons states (n_electrons is the number of electrons). This means that the Hamiltonian is already represented in the Hartree-Fock basis. States are Hartree-Fock solutions.

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np


import matplotlib.pyplot as plt

import math

In [2]:
def hydrogen_hamiltonian(coordinates):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    molecule = qml.qchem.Molecule(["H", "H"], coordinates)
    return qml.qchem.molecular_hamiltonian(
        molecule
    )[0]


In [17]:
hydrogen_hamiltonian(np.array([0.0, 0.0, -0.8, 0.0, 0.0, 0.8]))


(
    -0.21359179710648957 * I(0)
  + 0.1560349960153825 * Z(0)
  + 0.15603499601538245 * Z(1)
  + 0.16362674506422686 * (Z(0) @ Z(1))
  + -0.17937702469836225 * Z(2)
  + 0.11457952445808016 * (Z(0) @ Z(2))
  + 0.1614207222409031 * (Z(1) @ Z(2))
  + 0.04684119778282293 * (Y(0) @ X(1) @ X(2) @ Y(3))
  + -0.04684119778282293 * (Y(0) @ Y(1) @ X(2) @ X(3))
  + -0.04684119778282293 * (X(0) @ X(1) @ Y(2) @ Y(3))
  + 0.04684119778282293 * (X(0) @ Y(1) @ Y(2) @ X(3))
  + -0.17937702469836225 * Z(3)
  + 0.1614207222409031 * (Z(0) @ Z(3))
  + 0.11457952445808016 * (Z(1) @ Z(3))
  + 0.16964861601025572 * (Z(2) @ Z(3))
)

In [14]:
def hf(num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.

    Args:
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
    
    return qml.qchem.hf_state(2, num_qubits)


In [15]:
hf(4)

array([1, 1, 0, 0])

In [20]:
def run_VQE(coordinates):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """

    hamiltonian = hydrogen_hamiltonian(np.array(coordinates))

    num_qubits = len(hamiltonian.wires)

    hf_state = hf(num_qubits)
    # singles and doubles are used to make the AllSinglesDoubles template
    singles, doubles = qml.qchem.excitations(2, num_qubits)

    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def cost(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.

        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
        """
         # Put your solution here #
        wires = range(num_qubits)
        qml.BasisState(hf_state, wires = num_qubits)
        qml.templates.AllSinglesDoubles(weights, wires, hf_state, singles, doubles)
        return qml.expval(hamiltonian)
    
    
    np.random.seed(1234)
    weights = np.random.normal(
        0, np.pi, len(singles) + len(doubles), requires_grad=True
    )
    opt = qml.AdamOptimizer(0.5)

    for _ in range(200):
        weights = opt.step(cost, weights)

    return cost(weights)

In [21]:
coordinates = [0.0, 0.0, -0.8, 0.0, 0.0, 0.8]
run_VQE(coordinates)

ValueError: State must be of length 1; got length 4 (state=[1 1 0 0]).